追従実験考えてみる
1. ユーザーをsimhashでいい感じにコホートに分解できるのか
2. コホートの性質の分析
3. コホートからリバースエンジニアリングできるか？

1. ユーザーをsimashでいい感じにコホートに分解できるのか  
シンプルなoneHotで試してみる．

In [2]:
import numpy as np
import pandas as pd

In [44]:
d = 1000
n = 10000
root_cohort_num = 100 # 大カテゴリ， Web上の履歴に大きく影響を及ぼす(性別とか年齢とか)
p_root_cohort = 0.01
week_cohort_num = 1000 # 少カテゴリ， 小さい確率でそれっぽい行動をする(個人端末で仕事のこと調べるとか)
p_week_cohort = 0.005
p_noize = 0.001

In [45]:
cohort_roots = [np.random.binomial(100, p_root_cohort, d) for i in range(root_cohort_num)]
cohort_week = [np.random.binomial(100, p_week_cohort, d) for i in range(week_cohort_num)]
result = []
for i in range(0, n):
  # 誤差乗せる
  root_idx = i % root_cohort_num
  week_idx = i % week_cohort_num
  result.append(cohort_roots[root_idx] + cohort_week[week_idx] + np.random.binomial(1, p_noize, d)) 
  #誤差 ０.01だとそこそこブレる， まぁぶれたならそこが大事だからちゃんと反映されるねという話はある
  #よく考えたらカラムはドメインなので， 0.01 * dした数だけ違うドメインに飛んでたらそれは違うクラスタだわ

In [46]:
weights = [np.random.randint(-1, 2, d) for i in range(100)] # 適当に100bitでやってみる

In [47]:
vecs = []
for i in range(n):
  vec = []
  for weight in weights:
    vec.append(1 if np.dot(result[i], weight) >= 0 else 0)
  vecs.append(vec)

In [48]:
bites = [bidnizer(each) for each in vecs]

In [49]:
def bidnizer(tgt : list):
  out = 0
  for each in tgt:
    out = (out << 1) | each
  return out

In [50]:
cohort_id = []
week_cohort_id = []
for i in range(0, n):
  root_idx = i % root_cohort_num
  week_idx = i % week_cohort_num
  cohort_id.append(root_idx)
  week_cohort_id.append(week_idx)

In [51]:
tmpDf = pd.DataFrame(list(zip(bites, cohort_id, week_cohort_id)), columns = ["bite", "cohort_id", "week_cohort_id"])

In [52]:
sTmpDf = tmpDf.sort_values("bite")

In [53]:
pd.set_option("max_rows", 100000)

In [54]:
sTmpDf
# cohort_idの並びを見るとそこそこできている…のか？
# 弱いコホートに関してもまぁある程度集約できてる
# ノイズによって変わるけど十分大きくするとメモリが足りなくなって面倒なのでココで終わる

,bite,cohort_id,week_cohort_id
3015,36270904142793845317066636999,15,15
9015,36270904142793845317066636999,15,15
1015,36270904142793845317066636999,15,15
7015,36270904142793845317066636999,15,15
4015,36270904142793845317066636999,15,15
8015,36270904142793845317066636999,15,15
15,36270904142793845317066636999,15,15
5015,36270904142793845317066636999,15,15
6015,36270904142793845317066636999,15,15
5353,36307095008857487227905729989,53,353
